<a href="https://colab.research.google.com/github/Pedrohero13/DB_NO_CON_colab/blob/main/Employees.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Instalación de la librería streamlit

In [1]:
!pip install streamlit


Vamos a cargar una máquina virtual que tenga instalado Ngrok

In [2]:
!wget https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-linux-amd64.zip

--2022-03-02 22:24:39--  https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-linux-amd64.zip
Resolving bin.equinox.io (bin.equinox.io)... 54.237.133.81, 54.161.241.46, 18.205.222.128, ...
Connecting to bin.equinox.io (bin.equinox.io)|54.237.133.81|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 13832437 (13M) [application/octet-stream]
Saving to: ‘ngrok-stable-linux-amd64.zip’

ngrok-stable-linux- 100%[===================>]  13.19M  2.80MB/s    in 11s     

2022-03-02 22:24:51 (1.17 MB/s) - ‘ngrok-stable-linux-amd64.zip’ saved [13832437/13832437]



Ahora cargaremos esta máquina virtual en nuestro archivo de Google Colab

In [3]:
!unzip ngrok-stable-linux-amd64.zip

Archive:  ngrok-stable-linux-amd64.zip
  inflating: ngrok                   


Add token ngrok to ngrok suite

In [79]:
get_ipython().system_raw('./ngrok authtoken 24qeTD0luOCmIfQYHBvCGHHddbz_5CF1HTN82FtBvpFrd8yhf')

Authtoken saved to configuration file: /root/.ngrok2/ngrok.yml


Posteriormente le indicaremos a Ngrok que estará utilizando el puerto 8501 para visualizar nuestra aplicación

In [102]:
get_ipython().system_raw('./ngrok http 8501 &')

Una vez inicializado el puerto, ahora vamos a crear el tunel para que pueda generarse un enlace que nos permita visualizar nuestra aplicación

In [103]:
!curl -s http://localhost:4040/api/tunnels | python3 -c \
    'import sys, json; print("Execute the next cell and the go to the following URL: " +json.load(sys.stdin)["tunnels"][0]["public_url"])'

Execute the next cell and the go to the following URL: http://988c-35-221-208-120.ngrok.io


Una vez que ejecutaron la instrucción anterior, verán que aparece un URL que es el que vamos a dar clic para visualizar nuestra aplicación, pero primero tenemos que ejecutar nuestra aplicación

# **RETO: Análisis de deserción de empleados**

Ahora creamos nuestra aplicacion, para guardar el archivo nos aseguramos de incluir la sentencia: %%writefile employees.py

In [154]:
%%writefile employees.py
import pandas as pd
import numpy as np
import streamlit as st
import codecs
import matplotlib.pyplot as plt
import re

st.title('Empleados')
st.header ("Análisis de deserción de empleados")

st.write ("""
Esta aplicación analiza la deserción de empleados de una empresa
""")


sidebar = st.sidebar
sidebar.title("MENU")


DATA_URL = ('/content/employees.csv')

@st.cache
def load_data(nrows):
    data = pd.read_csv(DATA_URL, nrows=nrows, encoding_errors='ignore')
    return data
data = load_data(1000) #cache para las funciones 
#Cargar datos si se selecciona en el checkbox
agree = sidebar.checkbox("¿Quieres mostrar todos los datos? ",key ="Dataframe")
if agree:
  data_load_state = st.text('Cargando...')
  data = load_data(1000)
  
  data_load_state.text("Cargado! (usando st.cache)")
  st.dataframe(data)
  
sidebar.markdown("___")
#filtrar por employees_ID hometown o unit
@st.cache
def load_data_byEmployeesID(id):
  
  filtered_data_byname = data[data['Employee_ID'].str.contains(id,flags=re.IGNORECASE)]
  
  return filtered_data_byname

@st.cache
def load_data_byHometown(hometown):
  
  filtered_data_byname = data[data['Hometown'].str.contains(hometown,flags=re.IGNORECASE)]
  
  return filtered_data_byname

@st.cache
def load_data_byUnit(unit):
  
  filtered_data_byname = data[data['Unit'].str.contains(unit, flags=re.IGNORECASE)]
  return filtered_data_byname

inputUser = st.sidebar.text_input('Ingrese el id o ciudad o el trabajo: ')
btnFilteredID = sidebar.button('Buscar por ID')
btnFilteredHometown = sidebar.button('Buscar por Ciudad natal')
btnFilteredUnit = sidebar.button('Buscar por Trabajo')

if (btnFilteredID):
  st.write ("ID buscado: "+ inputUser)
  filterbyname = load_data_byEmployeesID(inputUser)
  count_row = filterbyname.shape[0]
  st.write(f'Total: {count_row}')

  st.dataframe(filterbyname)

if (btnFilteredHometown):
  st.write ("Ciudad buscada: "+ inputUser)
  filterbyname = load_data_byHometown(inputUser)
  count_row = filterbyname.shape[0]
  st.write(f'Total: {count_row}')

  st.dataframe(filterbyname)

if (btnFilteredUnit):
  st.write ("Trabajo buscado: "+ inputUser)
  filterbyname = load_data_byUnit(inputUser)
  count_row = filterbyname.shape[0]
  st.write(f'Total: {count_row}')

  st.dataframe(filterbyname)

sidebar.markdown("___")

#Filtrar por nivel educatico por selectBox
@st.cache
def load_data_bydire(level):
  filtered_data_byLevel = data[data['Education_Level'] == level]
  return filtered_data_byLevel

selected = sidebar.selectbox("Selecciona el Nivel educativo", data['Education_Level'].unique())
btnFilterByLevel = sidebar.button('Filtrar por Nivel educativo')

if (btnFilterByLevel): 
  st.write("Empleados con nivel educativo "+ str(selected))
  filterbylevel = load_data_bydire(selected)
  count_row = filterbylevel.shape[0]
  st.write(f'Total: {count_row}')

  st.dataframe(filterbylevel)
sidebar.markdown("___")
#Filtrar por Ciudad natal por selectBox
@st.cache
def load_data_byhome(home):
  filtered_data_byHome = data[data['Hometown'] == home]
  return filtered_data_byHome

selectedHome = sidebar.selectbox("Selecciona la ciudad natal", data['Hometown'].unique())
btnFilterByHometown = sidebar.button('Filtrar por Ciudad')

if (btnFilterByHometown): 
  st.write("Empleados con Ciudad natal "+ str(selectedHome))
  filterbyhome = load_data_byhome(selectedHome)
  count_row = filterbyhome.shape[0]
  st.write(f'Total: {count_row}')

  st.dataframe(filterbyhome)
sidebar.markdown("___")
#Filtrar por Tipo de trabajo por selectBox
@st.cache
def load_data_byUnit(unit):
  filtered_data_byunit = data[data['Unit'] == unit]
  return filtered_data_byunit

selectedUnit = sidebar.selectbox("Selecciona el trabajo", data['Unit'].unique())
btnFilterByUnit = sidebar.button('Filtrar por Trabajo')

if (btnFilterByUnit): 
  st.write("Empleados con el trabajo "+ str(selectedUnit))
  filterbyunit = load_data_byUnit(selectedUnit)
  count_row = filterbyunit.shape[0]
  st.write(f'Total: {count_row}')

  st.dataframe(filterbyunit)
sidebar.markdown("___")

sidebar.subheader("Graficas")
# histograma edades 
agreeHis = sidebar.checkbox("Histograma de edades ",key = "edades")
if agreeHis:
  fig, ax = plt.subplots()

  ax.hist(data['Age'], rwidth=0.85)
  ax.set_xlabel("Edad")
  ax.set_ylabel("Numero de empleados")
  st.header("Histograma de empleados por edad")

  st.pyplot(fig)

  st.markdown("___")

# frecuencia  
agreeFrecuency = sidebar.checkbox("Frecuencia por trabajo ",key = "frecuencia")
if agreeFrecuency:
  fig, ax = plt.subplots()

  ax.hist(data['Unit'], rwidth=0.85)
  ax.set_xlabel("Trabajo")
  ax.set_ylabel("Numero de empleados")
  st.header("Frecuencia de Empleados por Trabajo ")
  plt.setp(ax.get_xticklabels(), rotation=45, ha='right')
  st.pyplot(fig)
  
  st.markdown("___")

# indice de desercion por ciudad
agreeDesercion = sidebar.checkbox("Indice de deserción por ciudad ",key = "desercion")
if agreeDesercion:
  fig, ax = plt.subplots()

  y_pos = data['Attrition_rate']
  x_pos = data['Hometown']

  ax.bar(x_pos, y_pos)
  ax.set_ylabel("Desercion")
  ax.set_xlabel("Ciudad natal")
  ax.set_title('¿Cuantos empleados desertaron por ciudad?')

  st.header("Indice de deserción por ciudad")

  st.pyplot(fig)

  st.markdown("___")

# indice de desercion por edad

agreeEdad = sidebar.checkbox("Indice de deserción por edad ", key = "desEdad")
if agreeEdad:
  fig, ax = plt.subplots()

  y_pos = data['Attrition_rate']
  x_pos = data['Age']

  ax.barh(x_pos, y_pos)
  ax.set_xlabel("Desercion")
  ax.set_ylabel("Edad")
  ax.set_title('¿Cuantos empleados desertaron por edad?')

  st.header("Indice de deserción por edad")

  st.pyplot(fig)

  st.markdown("___")

# indice de desercion por tiempo de servicio

agreeService = sidebar.checkbox("Indice de deserción por Tiempo de servicio ",key = "service")
if agreeService:
  fig, ax = plt.subplots()

  y_pos = data['Attrition_rate']
  x_pos = data['Time_of_service']

  ax.bar(x_pos, y_pos)
  ax.set_ylabel("Desercion")
  ax.set_xlabel("Tiempo de servicio")
  ax.set_title('¿Cuantos empleados desertaron por tiempo de servicio?')

  st.header("Indice de deserción por tiempo de servicio")

  st.pyplot(fig)

  st.markdown("___")









Overwriting employees.py


Una vez cargado y ejecutado el archivo ahora vamos a proceder a ejecutarlo con el comando streamlit para que se visualice el contenido del mismo.

In [155]:
get_ipython().system_raw('./ngrok http 8501 &')

In [156]:
!curl -s http://localhost:4040/api/tunnels | python3 -c \
    'import sys, json; print("Execute the next cell and the go to the following URL: " +json.load(sys.stdin)["tunnels"][0]["public_url"])'

Execute the next cell and the go to the following URL: https://2334-35-221-208-120.ngrok.io


In [ ]:
!streamlit run /content/employees.py

2022-03-03 01:42:17.187 INFO    numexpr.utils: NumExpr defaulting to 2 threads.

  You can now view your Streamlit app in your browser.

  Network URL: http://172.28.0.2:8501
  External URL: http://35.221.208.120:8501

